# Estimating whether a stock will go up, down, or hold based on news headlines

We will use a Random Forest Classifier, a Naive-Bayes classifier with a "bag of words" approach, and an LSTM Recurrent Neural Network to see which gives us the best performance.

LABELS: 

For our initial approaches, prior to trying our neural network's hand at this problem, we'll be using a simpler binary classification, where: 

0 -> Stocks will go down

1 -> Stocks will go up or hold

In [1]:
# Import numpy and pandas to start
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read in our first dataset as df
df = pd.read_csv('/content/drive/MyDrive/StockStalker/Data/Stock_Dataa.csv', encoding='ISO-8859-1')

In [4]:
# Look at first 5 rows
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United (England),Women in record South Pole walk,Vasco da Gama (Brazil),South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,Revenge is sweet for jubilant Cronje,"Our choice, not theirs",Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,"Thuggery, Toxins and Ties",Met faces fresh attack on race crime,Everton fans top racist 'league of shame',"Our breasts, ourselves",Russia's new boss has an extremely strange his...,Always and forever,Most everywhere: UDIs,Most wanted: Chloe lunettes,

In [5]:
# Look at last 5 rows
df.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,Weapons shipped into Jordan by the CIA and Sau...,Angela Merkel said the U.K. must file exit pap...,In a birth offering hope to a threatened speci...,Sky News Journalist Left Speechless As Leave M...,Giant panda in Macau gives birth to twins,Get out now: EU leader tells Britain it must i...,Sea turtle 'beaten and left for dead' on beach...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
4097,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,UK's Osborne: 'Absolutely' going to have to cu...,'Do not let Scotland down now' : Scottish MEP ...,British pound could hit history-making dollar ...,"Merkel vows to strengthen EU, tells UK no 'che...","""Ryanair will not deploy new aircraft on route...","People, ever more greedy and stupid, destroy t...",Siemens freezes new UK wind power investment f...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
4098,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,Iceland historian Johannesson wins presidentia...,99-Million-Yr-Old Bird Wings Found Encased in ...,A chatbot programmed by a British teenager has...,The Philippine president-elect said Monday he ...,Former Belgian Prime Minister ridicules Nigel ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Islamic State bombings in southern Yemen kill ...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
4099,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says p

Data preprocessing for our text data:

In [6]:
# Remove punctuation
data = df.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Also, I'm going to rename our columns for ease of access
list1 = [i for i in range(25)]
new_index = [str(i) for i in list1]
data.columns = new_index
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,Hammers hand Robson a youthful lesson,Saints party like it s,Wear wolves have turned into lambs,Stump mike catches testy Gough s taunt,Langer escapes to hit,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,A tale of two tails,I say what I like and I like what I say,Elbows Eyes and Nipples,Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite
2,Coventry caught on counter by Flo,United s rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United England,Women in record South Pole walk,Vasco da Gama Brazil,South Melbourne Australia,Necaxa Mexico,Real Madrid Spain,Raja Casablanca Morocco,Corinthians Brazil,Tony s pet project,Al Nassr Saudi Arabia,Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,Auntie back in fashion even if the dress look...,Shoaib appeal goes to the top,Hussain hurt by shambles but lays blame on e...,England s decade of disasters,Revenge is sweet for jubilant Cronje,Our choice not theirs,Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for,Christmas glitches,Upending a table Chopping a line and Scoring ...,Scientific evidence unreliable defence claims,Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake up of failing NHS,Lessons of law s hard heart
4,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers are you all whingers,Hollywood Beyond,Ashes and diamonds,Whingers a formidable minority,Alan Parker part two,Thuggery Toxins and Ties,Met faces fresh attack on race crime,Everton fans top racist league of shame,Our breasts ourselves,Russia s new boss has an extremely strange his...,Always and forever,Most everywhere UDIs,Most wanted Chloe lunettes,Return of the cane completely off the agenda,From Sleepy Hollow to Greeneland,Blunkett outlines vision for over s,Embattled Dobson attacks play now pay later ...,Doom and the Dome,What is the north south divide,Aitken released from jai

In [7]:
# Converting the headlines into lowercase
# so all words are treated the same (apple and Apple, for example)
for index in new_index:
  data[index] = data[index].str.lower()
data.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [8]:
# Combining the first 25 headlines of first record
' '.join(str(x) for x in data.iloc[1,0:25])

'scorecard the best lake scene leader  german sleaze inquiry cheerio  boyo the main recommendations has cubie killed fees  has cubie killed fees  has cubie killed fees  hopkins  furious  at foster s lack of hannibal appetite has cubie killed fees  a tale of two tails i say what i like and i like what i say elbows  eyes and nipples task force to assess risk of asteroid collision how i found myself at last on the critical list the timing of their lives dear doctor irish court halts ira man s extradition to northern ireland burundi peace initiative fades after rebels reject mandela as mediator pe points the way forward to the ecb campaigners keep up pressure on nazi war crimes suspect jane ratcliffe yet more things you wouldn t know without the movies millennium bug fails to bite'

In [9]:
# Combining the headlines for the rest of our records so we can convert them into vectors
headlines = []
for row in range(0, len(data.index)):
  headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))


df['Headlines'] = headlines

data = df[['Label', 'Headlines']]
data

,Label,Headlines
0,0,a hindrance to operations extracts from the...
1,0,scorecard the best lake scene leader german s...
2,0,coventry caught on counter by flo united s riv...
3,1,pilgrim knows how to progress thatcher facing ...
4,1,hitches and horlocks beckham off but united su...
...,...,...
4096,0,barclays and rbs shares suspended from trading...
4097,1,scientists to australia if you want to ...
4098,1,explosion at airport in istanbul yemeni former...
4099,1,jamaica proposes marijuana dispensers for tour...


In [10]:
from sklearn.model_selection import train_test_split

# Set our X and y
# X -> features
# y -> target

X = data['Headlines']
y = data["Label"]

print(len(data))
print(len(y))

# Perform our train test split using sklearn's train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(len(X_train))
print(len(y_train))

4101
4101
3075
3075


## Natural Language Processing

Using 'bag of words' model and a TF-IDF Vectorizer for converting text data into vectors. 

For both Random Forest Classifier & Naive-Bayes Classifier

In [11]:
# Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [12]:
# Implementing Bag of Words model: 
countvector = CountVectorizer(ngram_range=(2, 2))
train_dataset = countvector.fit_transform(X_train) # Converting all the headlines to vectors

In [13]:
# Data converts into sparse matrix 
train_dataset[0]

<1x477881 sparse matrix of type '<class 'numpy.int64'>'
	with 416 stored elements in Compressed Sparse Row format>

First, we'll use our bag of words model with Random Forest Classifier

In [14]:
# Implement random forest classifier
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_dataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
# Predicting for our test dataset
test_dataset = countvector.transform(X_test)
predictions= random_classifier.predict(test_dataset)

In [16]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [17]:
# For calculating our accuracy
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [18]:
matrix= confusion_matrix(y_test,predictions)
print(matrix)
score= accuracy_score(y_test,predictions)
print(score)
report= classification_report(y_test,predictions)
print(report)

[[ 34 458]
 [ 33 501]]
0.5214424951267057
              precision    recall  f1-score   support

           0       0.51      0.07      0.12       492
           1       0.52      0.94      0.67       534

    accuracy                           0.52      1026
   macro avg       0.51      0.50      0.40      1026
weighted avg       0.52      0.52      0.41      1026



It looks like our overall accuracy is around 51% with this approach. 

The notebook I based my initial preprocessing and approach off (cited below, Sun, J.) acheived a nearly 84% accuracy with this approach, although I've split my test and train data somewhat differently. 

In Sun, J.'s approach, the test and train data actually overlapped quite a bit: for the entire year 2015. My concern is with having a model overfit in our production case, and so I opted to use the SciKit Learn's built in train_test_split function. 

In the future, after experimenting with other models and architectures, I will come back and potentially explore other ways of optimizing these results.

# Using Random Forest Classfier with TF-IDF Vectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [20]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
train_dataset= tfidf.fit_transform(X_train) # Changing all our headlines to vectors using TF-IDF technique

In [21]:
# Implement random forest classfier on train_dataset
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_dataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
# Predict on test dataset
test_dataset= tfidf.transform(X_test)
predictions= random_classifier.predict(test_dataset)

In [23]:
predictions

array([0, 1, 0, ..., 1, 1, 1])

In [24]:
# ACCURACY AFTER USING TF-IDF VECTORIZER
matrix= confusion_matrix(y_test,predictions)
print(matrix)
score= accuracy_score(y_test,predictions)
print(score)
report= classification_report(y_test,predictions)
print(report)

[[124 368]
 [118 416]]
0.5263157894736842
              precision    recall  f1-score   support

           0       0.51      0.25      0.34       492
           1       0.53      0.78      0.63       534

    accuracy                           0.53      1026
   macro avg       0.52      0.52      0.48      1026
weighted avg       0.52      0.53      0.49      1026



It still looks like our accuracy is quite low, and further optimizations will be required with these approaches.

However, first, we're going to try our Naive-Bayes Classifier and see how that performs, and then we're going to examine another architecture that may be better for this particular sentiment analysis problem: an LSTM.

## Naive-Bayes Classifier

First, we'll use our 'bag of words' model to convert text into vectors.

In [25]:
from sklearn.naive_bayes import MultinomialNB
naive = MultinomialNB()

In [26]:
countvector = CountVectorizer(ngram_range=(2, 2))
train_dataset = countvector.fit_transform(X_train)

In [27]:
# Fitting Naive Bayes Classifier with training data
naive.fit(train_dataset, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
# Predicting on test dataset
test_dataset = countvector.transform(X_test)
predictions = naive.predict(test_dataset)

In [29]:
predictions

array([0, 1, 1, ..., 1, 1, 0])

In [30]:
matrix= confusion_matrix(y_test, predictions)
print(matrix)
score= accuracy_score(y_test, predictions)
print(score)
report= classification_report(y_test, predictions)
print(report)

[[130 362]
 [121 413]]
0.5292397660818714
              precision    recall  f1-score   support

           0       0.52      0.26      0.35       492
           1       0.53      0.77      0.63       534

    accuracy                           0.53      1026
   macro avg       0.53      0.52      0.49      1026
weighted avg       0.53      0.53      0.50      1026



We could try Naive-Bayes with TF-IDF Vectorization, but I'm not convinced that it's necessarily going to perform any better for this problem. I'd like to take a different approach. First, I want to look at LSTM RNN's, and see how one performs on this dataset.

Knowing that LSTM's often need a LOT of data to perform well, I'm then going to see if I can combine the data that I have, preprocess it accordingly, and provide the LSTM with the combined set so it's better able to train. 

First, though, I'm going to just 'prove concept' with the current dataset that we're working with, and see if our accuracy even marginally improves. It's possible another type of architecture altogether suits us better, so let's see!

# LSTM with bag of words model & limited data

LABELS: 

0 -> Stock will go down

1 -> Stock will go up or hold

In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import numpy as np
import re

In [ ]:
# Set parameters

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)
X = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(X)

In [ ]:
# Compose the LSTM network.
# embed_dim, lstm_out, batch_size, and dropout_x are hyperparameters that may need to be tweaked in order to optimize output
# input_length, which we pass to our Embedding layer, takes the 1st index of our X value -> or, the length of our features?
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 537, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 537, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
print(type(X))
print(type(y_train))
model.fit(X, y_train, epochs = 15, batch_size=batch_size, verbose = 2)

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
Epoch 1/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 2/15
97/97 - 304s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 3/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 4/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 5/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 6/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 7/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 8/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 9/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 10/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 11/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 12/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 13/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 14/15
97/97 - 307s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 15/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265


# LSTM with TF-IDF Vectorization and increased data

### Using by-company stock data scraped from Reddit & Yahoo Finance

LABELS: 

0 -> Stock will go down (negative)

1 -> Stock will go up (positive)

2 -> Stock will hold (neutral)

In [35]:
# Get data & perform sentiment analysis to provide training labels
djia_df = pd.read_csv("/content/drive/MyDrive/StockStalker/djia_news.csv")
nasdaq_df = pd.read_csv('/content/drive/MyDrive/StockStalker/nasdaq.csv')

print(djia_df)
print(nasdaq_df)

df = pd.concat([djia_df, nasdaq_df], ignore_index=True)
df

      Label Ticker                                           Headline
0         0    MMM  Employer who stole nearly $3M in wages from 15...
1         1    MMM  Huge new Facebook data leak exposed intimate d...
2         0    MMM  A campaign has accelerated to turn a disused r...
3         1    MMM  Google launches global human trafficking helpl...
4         1    MMM  Over 3m Saudi Women Don’t Have ID Cards; Saudi...
...     ...    ...                                                ...
2376      0    WMT  Walmart dumps e-cigarettes: Largest store in U...
2377      0    WMT  Walmart makes a $16 billion bet on India's boo...
2378      0    WMT  Walmart raises minimum age to buy tobacco to 2...
2379      0    WMT  Walmart Took Over Chile In Only Three Years An...
2380      2    WMT  Carla Cheney: Walmart Fired Me For Reporting D...

[2381 rows x 3 columns]
       Label Ticker                                           Headline
0          0      A  @TotesTravel : Airline shares tumble as New

,Label,Ticker,Headline
0,0,MMM,Employer who stole nearly $3M in wages from 15...
1,1,MMM,Huge new Facebook data leak exposed intimate d...
2,0,MMM,A campaign has accelerated to turn a disused r...
3,1,MMM,Google launches global human trafficking helpl...
4,1,MMM,Over 3m Saudi Women Don’t Have ID Cards; Saudi...
...,...,...,...
15557,1,ZNGA,Bitcoin Tops $1000 Again as Zynga Accepts Virt...
15558,1,ZNGA,Zynga Accepts Bitcoin For Microtransactions
15559,1,ZUMZ,Zumiez (ZUMZ) unusual put activity into earnin...
15560,1,ZUMZ,Zumiez Is Going Bankrupt


In [36]:
# Remove punctuation
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

df["Headline"] = df["Headline"].str.lower()

df["Headline"] = df["Ticker"] + ' ' + df["Headline"]

df.head(10)

,Label,Ticker,Headline
0,0,MMM,MMM employer who stole nearly m in wages fro...
1,1,MMM,MMM huge new facebook data leak exposed intima...
2,0,MMM,MMM a campaign has accelerated to turn a disus...
3,1,MMM,MMM google launches global human trafficking h...
4,1,MMM,MMM over m saudi women don t have id cards s...
5,1,MMM,MMM boris johnson promises tax cut for m high...
6,0,MMM,MMM canada spends m to stop female genital m...
7,1,MMM,MMM u s accuses china of detaining up to m x...
8,0,MMM,MMM more than m raised for humboldt broncos ...
9,0,MMM,MMM australian property giant meriton has been...


In [37]:
# One-hot encode the labels
from keras.utils.np_utils import to_categorical
labels = to_categorical(df["Label"], num_classes=3)

print(labels[:10])

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [38]:
# Split our train and test data

X = df.drop('Label', axis=1)
y = df['Label']
 
X_train, X_test, y_train, y_test = train_test_split(X, labels)

print(X_train.shape)
print(y_train.shape)

(11671, 2)
(11671, 3)


In [39]:
# Set parameters

max_features = 8000
tokenizer = Tokenizer(num_words=max_features, split=' ')
print(f"tokenizer: {tokenizer}")
tokenizer.fit_on_texts(X_train['Headline'])
X = tokenizer.texts_to_sequences(X_train['Headline'])
print(f"X size after to_sequences: {len(X)}")

X = pad_sequences(X)

print(X.shape)
print(y_train.shape)

tokenizer: <keras_preprocessing.text.Tokenizer object at 0x7fb7bc5f5d50>
X size after to_sequences: 11671
(11671, 66)
(11671, 3)


In [40]:
# Compose the LSTM network.
# embed_dim, lstm_out, batch_size, and dropout_x are hyperparameters that may need to be tweaked in order to optimize output
# input_length, which we pass to our Embedding layer, takes the 1st index of our X value -> or, the length of our features?
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 66, 128)           1024000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 66, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 1,279,391
Trainable params: 1,279,391
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# #find the maximum vocabulary size
# voc_size = (flows_scaled.max()+1).astype('int64')
# print(voc_size)

batch_size = 32
print(type(X))
print(type(y_train))
model.fit(X, y_train, epochs = 100, batch_size=batch_size, verbose = 2, validation_split=0.3)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Epoch 1/100
256/256 - 91s - loss: 0.7394 - accuracy: 0.6445 - val_loss: 0.7221 - val_accuracy: 0.6331
Epoch 2/100
256/256 - 85s - loss: 0.6238 - accuracy: 0.7058 - val_loss: 0.6910 - val_accuracy: 0.6513
Epoch 3/100
256/256 - 85s - loss: 0.4465 - accuracy: 0.8070 - val_loss: 0.7292 - val_accuracy: 0.6528
Epoch 4/100
256/256 - 87s - loss: 0.3182 - accuracy: 0.8704 - val_loss: 0.9194 - val_accuracy: 0.6682
Epoch 5/100
256/256 - 85s - loss: 0.2376 - accuracy: 0.9083 - val_loss: 0.9225 - val_accuracy: 0.6699
Epoch 6/100


Citations:

Sun, J. (2016, August). Daily News for Stock Market Prediction, Version 1. Retrieved (2021, March) from https://www.kaggle.com/aaron7sun/stocknews.